In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql import functions as F


In [39]:
spark = SparkSession.builder\
        .appName('transform')\
        .getOrCreate()

In [40]:
df= spark.read.parquet('../../data/processed/part-00000-93a63093-38e6-4c79-9597-cf30e93b080c-c000.snappy.parquet')
df.show()

+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|work_year|experience_level|employment_type|           job_title|salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---------+----------------+---------------+--------------------+------+---------------+-------------+------------------+------------+----------------+------------+
|     2024|              SE|             FT|         AI Engineer|202730|            USD|       202730|                US|           0|              US|           M|
|     2024|              SE|             FT|         AI Engineer| 92118|            USD|        92118|                US|           0|              US|           M|
|     2024|              SE|             FT|       Data Engineer|130500|            USD|       130500|                US|           0|              US|           M|
|     2024

In [41]:
columns_to_clean = [
    ('work_year', IntegerType()),
    ('experience_level', StringType()),
    ('job_title', StringType()),
    ('salary', DecimalType()),
    ('salary_currency', StringType()),
    ('salary_in_usd', DecimalType()),
    ('employee_residence', StringType()),
    ('remote_ratio', IntegerType()),
    ('company_location', StringType()),
    ('company_size', StringType())
]

clean_df = df

for column, data_type in columns_to_clean:
    clean_df = clean_df.withColumn(column, col(column).cast(data_type)) \
                       .filter(col(column).isNotNull())

In [42]:
clean_df.coalesce(1).write.parquet("../../data/final", mode='overwrite')